### RNA-seq data analysis
#### In this notebook we run a pipeline in which tools are integrated for RNA seq data analysis.
##### bowtie2: for alignment 
##### samtools: for sorting the sam file and coversion into bam file
##### stringtie: for transcript structure recovery and abundance estimation

In [1]:
!head data.txt

ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR861/008/SRR8615528/SRR8615528_1.fastq.gz
ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR861/008/SRR8615528/SRR8615528_2.fastq.gz


In [ ]:
# I save the ftp link of fastq file for data in a txt file and download using wget command 
!wget -i data.txt # -i option use in the case if you have multiple links for files in txt file 

### Quality Control using FastQC

In [1]:
#let's run command line fastqc for one sample 
!fastqc SRR13311153_1.fastq # it will produce a html output of quality report
# check the quality report and apply required steps for enhance the quality

Started analysis of SRR13311153_1.fastq
Approx 5% complete for SRR13311153_1.fastq
Approx 10% complete for SRR13311153_1.fastq
Approx 15% complete for SRR13311153_1.fastq
Approx 20% complete for SRR13311153_1.fastq
Approx 25% complete for SRR13311153_1.fastq
Approx 30% complete for SRR13311153_1.fastq
Approx 35% complete for SRR13311153_1.fastq
Approx 40% complete for SRR13311153_1.fastq
Approx 45% complete for SRR13311153_1.fastq
Approx 50% complete for SRR13311153_1.fastq
Approx 55% complete for SRR13311153_1.fastq
Approx 60% complete for SRR13311153_1.fastq
Approx 65% complete for SRR13311153_1.fastq
Approx 70% complete for SRR13311153_1.fastq
Approx 75% complete for SRR13311153_1.fastq
Approx 80% complete for SRR13311153_1.fastq
Approx 85% complete for SRR13311153_1.fastq
Approx 90% complete for SRR13311153_1.fastq
Approx 95% complete for SRR13311153_1.fastq
Analysis complete for SRR13311153_1.fastq


### Here we use 1st script that make bowtie2 index for reference genome 
### bowtie2

In [2]:
# bowtie2_index.sh require 2 arguments -t number of threads and -i input reference fasta for a genome
!bash bowtie2_index.sh -t 32 -i Homo_sapiens.GRCh38.dna.toplevel.fa > index_out.txt # reference should be with .fa extension
# bowtie2 build command produce large output on terminal so here i save the output in a txt file using greater then ">" sign

Building a LARGE index


### Here we perform bowtie2 alignment, samtools sorting and conversion into bam file and abundance estimation

## RNA-seq data analysis

In [2]:
# let's tail the index_out.txt to check about time
!tail index_out.txt

    sideSz: 128
    sideBwtSz: 96
    sideBwtLen: 384
    numSides: 8080749
    numLines: 8080749
    ebwtTotLen: 1034335872
    ebwtTotSz: 1034335872
    color: 0
    reverse: 1
Total time for backward call to driver() for mirror index: 00:24:16


In [1]:
# below program take forward and reverse reads, bowtie index prefix and gff3 file for annotation
!bash RNAseq_data_analysis.sh -1 SRR13311153_1.fastq -2 SRR13311153_2.fastq -r Homo_sapiens.GRCh38.dna.toplevel -g Homo_sapiens.GRCh38.106.gtf
# -1 take forward reads, -2 take reverse reads, -r take bowtie2 index prefix and -g take annotated reference gff3 file

Forward reads: SRR13311153_1.fastq
Reverse reads: SRR13311153_2.fastq
Refernce prefix: Homo_sapiens.GRCh38.dna.toplevel
Annotation file gff3: Homo_sapiens.GRCh38.106.gtf
Alignemt using bowtie2 for SRR13311153
26137510 reads; of these:
  26137510 (100.00%) were paired; of these:
    5498618 (21.04%) aligned concordantly 0 times
    11473351 (43.90%) aligned concordantly exactly 1 time
    9165541 (35.07%) aligned concordantly >1 times
    ----
    5498618 pairs aligned concordantly 0 times; of these:
      703659 (12.80%) aligned discordantly 1 time
    ----
    4794959 pairs aligned 0 times concordantly or discordantly; of these:
      9589918 mates make up the pairs; of these:
        6176631 (64.41%) aligned 0 times
        1953456 (20.37%) aligned exactly 1 time
        1459831 (15.22%) aligned >1 times
88.18% overall alignment rate
Sorting the Alignment file
[bam_sort_core] merging from 16 files and 16 in-memory blocks...
Stringtie quantification


In [3]:
# let's check the output file 
!head SRR13311153.gtf

# stringtie SRR13311153.bam -l SRR13311153 -p 16 -G Homo_sapiens.GRCh38.106.gtf -o SRR13311153.gtf
# StringTie version 2.1.1
1	StringTie	transcript	98865	99373	1000	.	.	gene_id "SRR13311153.1"; transcript_id "SRR13311153.1.1"; cov "5.145383"; FPKM "0.746613"; TPM "1.263307";
1	StringTie	exon	98865	99373	1000	.	.	gene_id "SRR13311153.1"; transcript_id "SRR13311153.1.1"; exon_number "1"; cov "5.145383";
1	StringTie	transcript	131025	134836	1000	+	.	gene_id "SRR13311153.2"; transcript_id "SRR13311153.2.1"; reference_id "ENST00000442987"; ref_gene_id "ENSG00000233750"; ref_gene_name "CICP27"; cov "0.590504"; FPKM "0.085684"; TPM "0.144982";
1	StringTie	exon	131025	134836	1000	+	.	gene_id "SRR13311153.2"; transcript_id "SRR13311153.2.1"; exon_number "1"; reference_id "ENST00000442987"; ref_gene_id "ENSG00000233750"; ref_gene_name "CICP27"; cov "0.590504";
1	StringTie	transcript	135141	135895	1000	-	.	gene_id "SRR13311153.3"; transcript_id "SRR13311153.3.1"; reference_id "ENST00000494149"; r

#### Repeat above process for all your samples and then make a txt file having gtf file name
##### for making txt file follow below concept

In [4]:
!ls SRR133111*.gtf > merge_gtf.txt

In [5]:
!cat merge_gtf.txt

SRR13311153.gtf
SRR13311154.gtf
SRR13311155.gtf
SRR13311156.gtf
SRR13311157.gtf
SRR13311158.gtf
SRR13311159.gtf
SRR13311160.gtf
SRR13311161.gtf
SRR13311162.gtf
SRR13311163.gtf
SRR13311164.gtf
SRR13311165.gtf
SRR13311166.gtf
SRR13311167.gtf
SRR13311168.gtf
SRR13311169.gtf
SRR13311170.gtf
SRR13311171.gtf
SRR13311172.gtf
SRR13311173.gtf
SRR13311174.gtf
SRR13311175.gtf
SRR13311176.gtf
SRR13311177.gtf
SRR13311178.gtf
SRR13311179.gtf
SRR13311180.gtf


### stringtie_ctab.sh
##### let's make output for ballgown package in r using above script

In [6]:
!bash stringtie_ctab.sh -l merge_gtf.txt -r Homo_sapiens.GRCh38.106 -s SRR13311178
# -l take txt file having gtf file name of sample
# -r take prefix name of gtf file 
# -s take sample that use for input bam file and make file gtf fil in ballgown folder

Forward reads: merge_gtf.txt
Reverse reads: Homo_sapiens.GRCh38.106
output file: Homo_sapiens.GRCh38.106.merge.gtf
